In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (validation_images, validation_labels) = fashion_mnist.load_data()

text_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = train_images/255.0
validation_images = validation_images/255.0
batchsize = 256
num_epochs = 100000
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
callbacks_list = [es]

In [ ]:
import pandas as pd
data = []
units = np.exp2(range(3,5)) #2,13
for h in units:
  for _ in range(20):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(h, activation='relu',kernel_initializer='glorot_normal'))
    model.add(tf.keras.layers.Dense(h, activation='relu',kernel_initializer='he_normal'))
    model.add(tf.keras.layers.Dense(10, activation='softmax',kernel_initializer='he_normal'))
    model.compile(optimizer=tf.keras.optimizers.Nadam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    entrenamiento = model.fit(x=train_images, y=train_labels, batch_size = batchsize, epochs=num_epochs,validation_data=(validation_images,validation_labels),callbacks=callbacks_list,verbose=0)
    train_res = model.evaluate(x=train_images, y=train_labels, batch_size =train_images.shape[0],verbose=0)
    test_res = model.evaluate(x=validation_images, y=validation_labels, batch_size =validation_images.shape[0],verbose=0)
    prediction = model.predict(x=validation_images)
    soft_acc = prediction[np.arange(len(prediction)), validation_labels].mean()
    pacman_gap = -np.log(soft_acc)-train_res[0]
    jensen_gap = test_res[0]+np.log(soft_acc)
    data.append({
            'Units': h,
            'Train Cross Entropy': train_res[0],
            'Hard Accuracy':  1-test_res[1],
            'Soft Accuracy': 1-soft_acc,
            'PACMAN Gap': pacman_gap,
            'Jensen Gap': jensen_gap
        })
  df = pd.DataFrame(data)
  print(df[df['Units']==h].mean())
  print(df[df['Units']==h].std())